In [36]:
import pandas as pd
import numpy as np
import os 
import re
pd.options.display.max_columns = None

Cargaremos los dos ficheros de datos

In [37]:
df_limp = pd.read_csv("attacks_limpieza_5.csv", index_col=0)
df_limp.head()

,case_number,type,country,area,location,activity,name,sex_,age,time,species_,href,fatal_(y/n),injury,date,sex_mode,type_mode,year,age_new
0,1800.00.00,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, all onboard were killed by sharks",1800,F,Unprovoked,1927.133059,27.267683
1,1797.05.28.R,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,FATAL,Reported May-28-1797,M,Unprovoked,1927.133059,27.267683
2,1792.09.12,Provoked,england,NaN,On the Thames River,Fishing,male,M,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, PROVOKED INCIDENT",12-Sep-1792,M,Provoked,1927.133059,27.267683
3,1791.00.00,Unprovoked,australia,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, ""bitten in two""",1791,F,Unprovoked,1927.133059,27.267683
4,1788.05.10,Boat,australia,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,N,"No injury to occupants, shark bit oar and rudder",10-May-1788,M,Boat,1927.133059,27.267683


In [38]:
df_etl = pd.read_csv("etl_1.csv", index_col = 0)
df_etl.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,-1,11,1012,none,1,0,100,3,USA
1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",27,2,11,1013,none,1,0,100,4,USA
2,9,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",27,-1,11,1013,none,1,0,95,3,USA
3,12,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",26,-1,11,1012,none,1,0,100,3,USA
4,15,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",27,-1,11,1013,none,1,0,100,3,USA


Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:
 - USA
 - Australia
 - New Zealand
 - South Africa
 - Papua New Guinea

In [30]:
filtrado_paises = df_limp[(df_limp['country'] == 'australia') 
                  |(df_limp['country'] == 'usa')|(df_limp['country'] == 'south africa') | (df_limp['country'] == 'papua new guinea')
                  | (df_limp['country'] == 'new zealand')]

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

In [31]:
pd.concat([filtrado_paises, df_etl])

,case_number,type,country,area,location,activity,name,sex_,age,time,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
3,1791.00.00,Unprovoked,australia,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1788.05.10,Boat,australia,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1780.08.08,Unprovoked,usa,Florida,"Pensacola Harbor, Escambia County",Swimming,Joseph Feron,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1779.00.00,Unprovoked,usa,Hawaii,"Maliu, Hawai'i",Surfing,Nu'u-anu-pa'a hu,M,young,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1771.07.12.R,Unprovoked,usa,NaN,Damiscotte,Fishing,male,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,15.0,13.0,1005.0,rain,4.0,0.0,20.0,5.0,Papua New Guinea
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,10.0,11.0,1006.0,none,4.0,0.0,5.0,5.0,Papua New Guinea
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,15.0,11.0,1007.0,none,4.0,0.0,350.0,5.0,Papua New Guinea
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,15.0,10.0,1009.0,rain,4.0,0.0,335.0,4.0,Papua New Guinea


In [ ]:
filtrado_paises.merge(df_etl, left_on=None, right_on=None, left_index=False, right_index=False)

In [35]:
pd.concat([filtrado_paises, df_etl])

,case_number,type,country,area,location,activity,name,sex_,age,time,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
3,1791.00.00,Unprovoked,australia,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1788.05.10,Boat,australia,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1780.08.08,Unprovoked,usa,Florida,"Pensacola Harbor, Escambia County",Swimming,Joseph Feron,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1779.00.00,Unprovoked,usa,Hawaii,"Maliu, Hawai'i",Surfing,Nu'u-anu-pa'a hu,M,young,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1771.07.12.R,Unprovoked,usa,NaN,Damiscotte,Fishing,male,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,15.0,13.0,1005.0,rain,4.0,0.0,20.0,5.0,Papua New Guinea
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,10.0,11.0,1006.0,none,4.0,0.0,5.0,5.0,Papua New Guinea
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,15.0,11.0,1007.0,none,4.0,0.0,350.0,5.0,Papua New Guinea
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,15.0,10.0,1009.0,rain,4.0,0.0,335.0,4.0,Papua New Guinea
